In [1]:
#importing required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
s = Service("C:\msedgedriver.exe")
driver = webdriver.Edge(service= s)
driver.get("https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/squads")
#page url we need to scrape
driver.maximize_window()

In [3]:
soup = BeautifulSoup(driver.page_source, 'lxml') #parsing the webpage

In [4]:
all_teams= []
data = soup.find_all("span",{"class": "ds-text-title-xs ds-font-bold ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block"})
for value in data:
    all_teams.append(value.text)
#collecting all team list  

In [5]:
all_teams

['Afghanistan Squad',
 'Australia Squad',
 'Bangladesh Squad',
 'England Squad',
 'India Squad',
 'Ireland squad',
 'Namibia Squad',
 'Netherlands Squad',
 'New Zealand Squad',
 'Pakistan Squad',
 'Scotland squad',
 'South Africa Squad',
 'Sri Lanka Squad',
 'United Arab Emirates Squad',
 'West Indies Squad',
 'Zimbabwe squad']

In [6]:
soup = BeautifulSoup(driver.page_source, 'lxml')
all_links = soup.find_all("a",{"class":"ds-inline-flex ds-items-start ds-leading-none"})
link_list= [i.get("href") for i in all_links]
test= [i.text for i in all_links]
link_list= link_list[0:16]
#creating list of links for each team which we will click to fetch each team's player data

In [7]:
player_name= []
player_team=[]
playing_role= []
batting_style= []
bowling_style= []

pattern= r"(?im)^(Age:)(?P<age>.*)(?:Batting\:\s+)(?P<bat_style>[\w\s]*)(?:BatBowling|Bat\:\s+)?(?P<bowl_style>.*)?"
#this pattern will allow us to seperate each player's age, batting style and bowling style
for link in link_list:
    misc_info= []
    new_url= driver.get("https://www.espncricinfo.com/"+link)
    time.sleep(2)
    new_soup = BeautifulSoup(driver.page_source, 'lxml')
    name= new_soup.find_all("span",{"class":"ds-text-title-m ds-font-bold ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block"})
    player_name.extend([i.text for i in name])
    role= new_soup.find_all("p",{"class":"ds-text-tight-l ds-mb-2 ds-mt-1"})
    playing_role.extend([i.text for i in role])
    other_info = new_soup.find_all("div",{"class":"ds-text-typo-mid3"})
    misc_info.extend([i.text for i in other_info])
    time.sleep(2)
    
    

    for text in misc_info:
        match = re.search(pattern, text)
        if match:
            batting_style.append(match.group("bat_style"))
            bowling_style.append(match.group("bowl_style"))
    for val in name:
        player_team.append(all_teams[link_list.index(link)])
           
    driver.back()    
    time.sleep(2)

In [8]:
player_info_df= pd.DataFrame()

In [9]:
player_info_df["Player Name"] = player_name
player_info_df["Team"] = player_team
player_info_df["Playing Role"] = playing_role
player_info_df["Batting Style"] = batting_style
player_info_df["Bowling Style"] = bowling_style


In [10]:
len(player_team)
len(batting_style)
len(bowling_style)

254

In [11]:
player_info_df.head(5)

,Player Name,Team,Playing Role,Batting Style,Bowling Style
0,Mohammad Nabi,Afghanistan Squad,Allrounder,Right hand BatBowling,: Right arm Offbreak
1,Azmatullah Omarzai,Afghanistan Squad,Allrounder,Right hand BatBowling,: Right arm Medium fast
2,Darwish Rasooli,Afghanistan Squad,Top order Batter,Right hand BatBowling,: Right arm Offbreak
3,Fareed Ahmad,Afghanistan Squad,Bowler,Left hand BatBowling,: Left arm Fast medium
4,Fazalhaq Farooqi,Afghanistan Squad,Bowler,Right hand BatBowling,: Left arm Fast medium


In [12]:
player_info_df["Batting Style"]= player_info_df["Batting Style"].apply(lambda x: x.replace("BatBowling","Bat"))
player_info_df["Bowling Style"]= player_info_df["Bowling Style"].apply(lambda x: x.strip(": "))
player_info_df["Team"]= player_info_df["Team"].apply(lambda x: x.replace("Squad",""))

In [13]:
player_info_df.head(5)

,Player Name,Team,Playing Role,Batting Style,Bowling Style
0,Mohammad Nabi,Afghanistan,Allrounder,Right hand Bat,Right arm Offbreak
1,Azmatullah Omarzai,Afghanistan,Allrounder,Right hand Bat,Right arm Medium fast
2,Darwish Rasooli,Afghanistan,Top order Batter,Right hand Bat,Right arm Offbreak
3,Fareed Ahmad,Afghanistan,Bowler,Left hand Bat,Left arm Fast medium
4,Fazalhaq Farooqi,Afghanistan,Bowler,Right hand Bat,Left arm Fast medium


In [14]:
#player_info_df.to_csv("C:\\Users\\Gurjeet\\Desktop\\End to End Cricket Analytics Project\\Webscraped Files\\dim_player_info.csv",encoding="utf-8-sig",index= False)